# Data Visualization with Altair / Vega

[Altair](https://altair-viz.github.io/) is a declarative statistical visualization library, based on the [Vega](http://vega.github.io/vega) and [Vega-Lite](http://vega.github.io/vega-lite) grammar of graphics.

You need to install the `altair` and `vega` Python packages, plus possibly `vega_dataset` if you want to work on their examples. Altair needs at least Python 3.5.3.

## Documentation

The [main documentation](https://altair-viz.github.io/) includes an extensive user guide as well as a gallery and case studies, here are a few links to special topics that are often needed but somewhat hard to find.

* [Mark definition parameters explained](https://altair-viz.github.io/user_guide/generated/core/altair.MarkDef.html#)
* [X axis specifications](https://altair-viz.github.io/user_guide/generated/channels/altair.X.html)

## Initialization

To register the Altair renderer in the classic notebook, you need the following code (typically in your first code cell).

In [5]:
import altair as alt

# Enable Altair for notebooks (not needed for JupyterLab)
_ = alt.renderers.enable('notebook')

Note that this is not necessary with JupyterLab.

To be able to render PNG images from Altair charts, you need some more prep work. Your Jupyter installation needs to have the `selenium` Python package installed (see the `setup` folder for hints on that), which in turn requires the `chromedriver` executable to work. 

The following code helps when you cannot install that program more globally and download it into the folder of your notebook, by adding it to the command search `PATH`.

In [4]:
import os

# Find a 'chromedriver' in the notebook's directory
if os.getcwd() not in os.environ['PATH'].split(os.pathsep): 
    os.environ['PATH'] += os.pathsep + os.getcwd()